In [14]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import QFT, Diagonal
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Kraus

# ============================================================
# PARAMETERS
# ============================================================
N_users = 4                      # number of users
n_qubits = int(np.log2(N_users)) # must satisfy N_users = 2^n
shots = 8192

#gamma = 0.2                      # amplitude damping
gamma = 0.2     
theta_strength = 0.1           # phase noise strength
#theta_strength = 0.3
# ============================================================
# 1️⃣ ENCODING
# |ψ> = (|0> + |N/2>)/√2
# ============================================================
qc = QuantumCircuit(n_qubits, n_qubits)

# Create |0> + |N/2>
qc.h(n_qubits - 1)

qc.barrier(label="Encoding")

# ============================================================
# 2️⃣ QFT (Multiplexing)
# ============================================================
qft = QFT(n_qubits, do_swaps=True)
qc.append(qft, range(n_qubits))
qc.barrier(label="QFT")

# ============================================================
# 3️⃣ NOISY CHANNEL
# ============================================================

# ---- 3.1 Phase distortion U_theta ----
theta = np.random.uniform(-theta_strength, theta_strength, N_users)

phase_unitary = Diagonal([
    np.exp(1j * t) for t in theta
])

qc.append(phase_unitary, range(n_qubits))

# ---- 3.2 Amplitude damping ----
K0 = np.array([[1, 0], [0, np.sqrt(1 - gamma)]])
K1 = np.array([[0, np.sqrt(gamma)], [0, 0]])

amp_damp = Kraus([K0, K1])

for q in range(n_qubits):
    qc.append(amp_damp, [q])

qc.barrier(label="Noisy Channel")

# ============================================================
# 4️⃣ IQFT (Demultiplexing)
# ============================================================
iqft = QFT(n_qubits, inverse=True, do_swaps=True)
qc.append(iqft, range(n_qubits))
qc.barrier(label="IQFT")

# ============================================================
# 5️⃣ MEASUREMENT
# ============================================================
qc.measure(range(n_qubits), range(n_qubits))

# ============================================================
# 6️⃣ SIMULATION
# ============================================================
sim = AerSimulator()
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=shots).result()
counts = result.get_counts()

print("Counts:", counts)

# ============================================================
# 7️⃣ QBER
# Correct output = |0...0>
# ============================================================
correct_state = "0" * n_qubits
p_correct = counts.get(correct_state, 0) / shots
qber = 1 - p_correct

print("QBER =", qber)

# ============================================================
# 8️⃣ SECURE KEY RATE (BB84)
# ============================================================
def h2(x):
    if x <= 0 or x >= 1:
        return 0
    return -x*np.log2(x) - (1-x)*np.log2(1-x)

key_rate = max(0, 1 - 2*h2(qber))
print("Secure key rate =", key_rate)


Counts: {'00': 3832, '10': 3864, '01': 223, '11': 273}
QBER = 0.5322265625
Secure key rate = 0


In [15]:
def compute_per_user_qber(counts, n_qubits):
    """
    counts   : dict from AerSimulator
    n_qubits : number of qubits (log2 N users)
    
    returns  : dict {user_index : QBER_u}
    """
    shots = sum(counts.values())
    N_users = 2**n_qubits

    qber_per_user = {}

    for u in range(N_users):
        # Expected output state for user u
        correct_state = format(u, f"0{n_qubits}b")

        p_correct = counts.get(correct_state, 0) / shots
        qber_per_user[u] = 1 - p_correct

    return qber_per_user


In [16]:
per_user_qber = compute_per_user_qber(counts, n_qubits)

for user, qber in per_user_qber.items():
    print(f"User {user}: QBER = {qber:.4f}")


User 0: QBER = 0.5322
User 1: QBER = 0.9728
User 2: QBER = 0.5283
User 3: QBER = 0.9667


In [17]:
total_qber = np.mean(list(per_user_qber.values()))
print("Total QBER =", total_qber)


Total QBER = 0.75


In [18]:
import numpy as np

def binary_entropy(x):
    x = np.clip(x, 1e-12, 1 - 1e-12)
    return -x*np.log2(x) - (1-x)*np.log2(1-x)


def compute_per_user_secure_key_rate(counts, n_qubits):
    """
    counts   : measurement counts from AerSimulator
    n_qubits : number of qubits (log2 N users)

    returns  : dict {user_index : secure_key_rate}
    """
    shots = sum(counts.values())
    N_users = 2**n_qubits

    key_rate_per_user = {}

    for u in range(N_users):
        correct_state = format(u, f"0{n_qubits}b")
        p_correct = counts.get(correct_state, 0) / shots
        qber_u = 1 - p_correct

        # BB84 secure key rate
        if qber_u >= 0.5:
            key_rate_per_user[u] = 0.0
        else:
            R_u = 1 - 2*binary_entropy(qber_u)
            key_rate_per_user[u] = max(0.0, R_u)

    return key_rate_per_user


In [19]:
per_user_key_rate = compute_per_user_secure_key_rate(counts, n_qubits)

for user, R in per_user_key_rate.items():
    print(f"User {user}: Secure key rate = {R:.4f}")


User 0: Secure key rate = 0.0000
User 1: Secure key rate = 0.0000
User 2: Secure key rate = 0.0000
User 3: Secure key rate = 0.0000


In [20]:
avg_key_rate = np.mean(list(per_user_key_rate.values()))
print("Average secure key rate =", avg_key_rate)


Average secure key rate = 0.0
